[Reference](https://blog.det.life/real-time-data-processing-pipeline-with-mongodb-kafka-debezium-and-risingwave-9575d48a897a)

# Analyzing Real-Time Data: The Pipeline


```
version: '3.1'

services:
  compactor-0:
    extends:
      file: ../../docker/docker-compose.yml
      service: compactor-0
  compute-node-0:
    extends:
      file: ../../docker/docker-compose.yml
      service: compute-node-0
  etcd-0:
    extends:
      file: ../../docker/docker-compose.yml
      service: etcd-0
  frontend-node-0:
    extends:
      file: ../../docker/docker-compose.yml
      service: frontend-node-0
  grafana-0:
    extends:
      file: ../../docker/docker-compose.yml
      service: grafana-0
  meta-node-0:
    extends:
      file: ../../docker/docker-compose.yml
      service: meta-node-0
  minio-0:
    extends:
      file: ../../docker/docker-compose.yml
      service: minio-0
  prometheus-0:
    extends:
      file: ../../docker/docker-compose.yml
      service: prometheus-0

  message_queue:
    extends:
      file: ../../docker/docker-compose.yml
      service: message_queue

  mongodb:
    image: mongo:4.4
    container_name: mongodb
    ports:
      - "27017:27017"
    command: --replSet rs0 --oplogSize 128
    restart: always
    healthcheck:
      test: "echo 'db.runCommand({ping: 1})' | mongo"
      interval: 5s
      timeout: 10s
      retries: 3

  mongodb-setup:
    image: mongo:4.4
    container_name: mongodb-setup
    depends_on:
      - mongodb
    entrypoint:
      [
        "bash",
        "-c",
        "sleep 10 && mongo --host mongodb:27017 /config-replica.js && sleep 10"
      ]
    restart: "no"
    volumes:
      - ./config-replica.js:/config-replica.js

  debezium:
    image: debezium/connect:1.9
    container_name: debezium
    depends_on:
      - message_queue
      - mongodb
    ports:
      - "8083:8083"
    healthcheck:
      test: curl -f localhost:8083
      interval: 1s
      start_period: 120s
    environment:
      BOOTSTRAP_SERVERS: message_queue:29092
      GROUP_ID: 1
      CONFIG_STORAGE_TOPIC: debezium_mongodb_config
      OFFSET_STORAGE_TOPIC: debezium_mongodb_offset
      STATUS_STORAGE_TOPIC: debezium_mongodb_status
      # Remove these two lines below to test the debezium json format。
      CONNECT_VALUE_CONVERTER: io.confluent.connect.avro.AvroConverter
      CONNECT_VALUE_CONVERTER_SCHEMA_REGISTRY_URL: http://message_queue:8081

  random_data_generator:
    build:
      context: .
      dockerfile: Dockerfile.generator
    container_name: random_data_generator
    depends_on:
      - mongodb
    environment:
      MONGO_HOST: mongodb
      MONGO_PORT: 27017
      MONGO_DB_NAME: random_data

  register-mongodb-connector:
    image: curlimages/curl:7.79.1
    container_name: register-mongodb-connector
    depends_on:
      debezium: { condition: service_healthy }
    command: >
      /bin/sh /register-mongodb-connector.sh
    restart: on-failure
    volumes:
      - ./register-mongodb-connector.sh:/register-mongodb-connector.sh

  kafka-connect-ui:
    image: landoop/kafka-connect-ui:0.9.7
    container_name: kafka-connect-ui
    depends_on:
      debezium: { condition: service_healthy }
    ports:
      - "8000:8000"
    environment:
      CONNECT_URL: http://debezium:8083

volumes:
  mongodb_data:
  message_queue:
    external: false
  debezium:
    external: false
```

In [1]:
docker compose up

In [2]:
psql -h localhost -p 4566 -d dev -U roota

In [3]:
CREATE TABLE users (_id JSONB PRIMARY KEY, payload JSONB) WITH (
    connector = 'kafka',
    kafka.topic = 'dbserver1.random_data.users',
    kafka.brokers = 'message_queue:29092',
    kafka.scan.startup.mode = 'earliest'
) ROW FORMAT DEBEZIUM_MONGO_JSON;

In [4]:
CREATE MATERIALIZED VIEW normalized_users AS
SELECT
    payload ->> 'name' as name,
    payload ->> 'email' as email,
    payload ->> 'address' as address
FROM
    users;

In [5]:
SELECT
    *
FROM
    normalized_users
LIMIT
    10;